In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.activations import swish
from tensorflow.keras.callbacks import TensorBoard
import time
import matplotlib.pyplot as plt
from matplotlib import gridspec

In [ ]:
NAME = "cifakeCNN{}".format(time.strftime("%Y%m%d-%H%M%S"))

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
#   try:
#     tf.config.set_logical_device_configuration(
#         gpus[0],
#         [tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Virtual devices must be set before GPUs have been initialized
#     print(e)

In [ ]:
# tf.config.threading.set_intra_op_parallelism_threads(1)
# tf.config.threading.set_inter_op_parallelism_threads(1)

In [ ]:
ds_train = image_dataset_from_directory(
    '/kaggle/input/cifar-10-fake-image-dataset/cifar-10-dataset/train',
    labels='inferred',
    label_mode='binary',
    image_size=[32,32],
    interpolation='nearest',
    batch_size=32,
    shuffle=True,
    seed=69,
    #validation_split=None,
    subset=None,
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
ds_test = image_dataset_from_directory(
    '/kaggle/input/cifar-10-fake-image-dataset/cifar-10-dataset/test',
    labels='inferred',
    label_mode='binary',
    image_size=[32,32],
    interpolation='nearest',
    batch_size=32,
    shuffle=True,
    seed=69,
    #validation_split=None,
    subset=None,
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE


In [ ]:
ds_train = (
    ds_train
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


In [ ]:
ds_valid = (
    ds_test
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=0.001,
    restore_best_weights=True)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=0.0008)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
timestamp = time.strftime("%Y%m%d-%H%M%S")
model_checkpoint = ModelCheckpoint(
    f'/kaggle/working/best/best_model_{timestamp}.keras', monitor='val_loss', save_best_only=True, mode='min'
)

In [ ]:
def MBConv6(input_tensor, hp=None, out_channels=32, expansion=6, use_se=True, dropout_rate=0.0):
    # Inverted Residual Structure
    # Pointwise Convolution  1 with Expansion Factor  6
    x = layers.Conv2D(expansion * input_tensor.shape[-1], (1,  1), padding='same', use_bias=False)(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    # Depthwise Separable Convolution with stride (1,  1)
    x = layers.DepthwiseConv2D(kernel_size=(3,  3), strides=(1,  1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    # Pointwise Convolution  2
    x = layers.Conv2D(out_channels, (1,  1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Dropout
    if dropout_rate >  0.0:
        x = layers.Dropout(dropout_rate)(x)

    # Bottleneck Convolution  
    x = layers.Conv2D(out_channels, (1,  1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Squeeze-and-Excitation
    if use_se:
        se_shape = (1,  1, out_channels)
        se = layers.GlobalAveragePooling2D()(x)
        se = layers.Reshape(se_shape)(se)
        se = layers.Conv2D(out_channels //  4, (1,  1), padding='same', use_bias=True)(se)
        se = layers.Activation(swish)(se)
        se = layers.Conv2D(out_channels, (1,  1), padding='same', use_bias=True)(se)
        se = layers.Activation('sigmoid')(se)
        x = layers.Multiply()([x, se])

    # Residual
    if input_tensor.shape[-1] == out_channels:
        shortcut = input_tensor
    else:
        shortcut = layers.Conv2D(out_channels, (1,  1), strides=(1,  1), padding='same', use_bias=False)(input_tensor)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])

    return x
input_tensor = layers.Input(shape=(32,  32,  16)) 
output_tensor = MBConv6(input_tensor)
model = keras.models.Model(inputs=input_tensor, outputs=output_tensor)

model.summary()

In [ ]:

def MBConv1(input_tensor, hp=None, expansion=4, out_channels=16, strides=(1,   1), use_se=True):
    # Expansion
    x = layers.Conv2D(expansion, (1,   1), padding='same', use_bias=False)(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    # Depthwise 
    x = layers.DepthwiseConv2D(kernel_size=(3,   3), strides=strides, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x) 

    # Pointwise 
    x = layers.Conv2D(out_channels, (1,   1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Squeeze-and-Excitation
    if use_se:
        se_shape = (1,  1, out_channels)
        se = layers.GlobalAveragePooling2D()(x)
        se = layers.Reshape((1,  1, out_channels))(se) 
        se = layers.Conv2D(out_channels //  2, (1,  1), padding='same', use_bias=True)(se) 
        se = layers.Activation(swish)(se)   
        se = layers.Conv2D(out_channels, (1,  1), padding='same', use_bias=True)(se) 
        se = layers.Activation('sigmoid')(se)
        se = layers.Reshape(se_shape[1:])(se)  # Reshape to original
        x = layers.Multiply()([x, se])

    # Residual
    if input_tensor.shape[-1] == out_channels and strides == (1,   1):
        x = layers.Add()([x, input_tensor])
    else:
        input_tensor = layers.Conv2D(out_channels, (1,   1), strides=strides, padding='same', use_bias=False)(input_tensor)
        x = layers.Add()([x, input_tensor])

    return x

input_tensor = layers.Input(shape=(32,  32,  3)) 
output_tensor = MBConv1(input_tensor)
model = keras.models.Model(inputs=input_tensor, outputs=output_tensor)

model.summary()

In [ ]:
import keras_tuner
from keras_tuner import RandomSearch, Hyperband
from tensorflow.keras.models import Model

def build_model(hp=None):
    activation_function = 'leaky_relu' if hp.Boolean(f'use_leaky_relu') else 'relu'
    input_tensor = layers.Input(shape=[32,  32,  3])
    
    x = layers.RandomRotation(factor=0.1488228133769420)(input_tensor)
    x = layers.RandomTranslation(height_factor=0.069, width_factor=0.069)(x)
    x = layers.RandomFlip(mode='horizontal')(x)
    x = layers.BatchNormalization()(x)
    

    x = MBConv1(x, hp=hp)
    x = MBConv6(input_tensor, hp=hp, out_channels=hp.Int('out_channels_1', min_value=20, max_value=44, step=12),
                expansion=hp.Int('expansion_1', min_value=4, max_value=8, step=2),
                use_se=hp.Boolean('use_se_1'),
                dropout_rate=hp.Float('conv6_dropout_rate_1', min_value=0.0, max_value=0.2, step=0.1))
    x = MBConv6(input_tensor, hp=hp, out_channels=hp.Int('out_channels_1', min_value=20, max_value=44, step=12),
                expansion=hp.Int('expansion_1', min_value=4, max_value=8, step=2),
                use_se=hp.Boolean('use_se_1'),
                dropout_rate=hp.Float('conv6_dropout_rate_1', min_value=0.0, max_value=0.2, step=0.1))
    x = MBConv6(input_tensor, hp=hp, out_channels=hp.Int('out_channels_1', min_value=20, max_value=44, step=12),
                expansion=hp.Int('expansion_1', min_value=4, max_value=8, step=2),
                use_se=hp.Boolean('use_se_1'),
                dropout_rate=hp.Float('conv6_dropout_rate_1', min_value=0.0, max_value=0.2, step=0.1))
    x = MBConv6(input_tensor, hp=hp, out_channels=hp.Int('out_channels_2', min_value=20, max_value=44, step=12),
                expansion=hp.Int('expansion_2', min_value=4, max_value=8, step=2),
                use_se=hp.Boolean('use_se_2'),
                dropout_rate=hp.Float('conv6_dropout_rate_2', min_value=0.0, max_value=0.2, step=0.1))
    x = MBConv6(input_tensor, hp=hp, out_channels=hp.Int('out_channels_2', min_value=20, max_value=44, step=12),
                expansion=hp.Int('expansion_2', min_value=4, max_value=8, step=2),
                use_se=hp.Boolean('use_se_2'),
                dropout_rate=hp.Float('conv6_dropout_rate_2', min_value=0.0, max_value=0.2, step=0.1))
    x = MBConv6(input_tensor, hp=hp, out_channels=hp.Int('out_channels_2', min_value=20, max_value=44, step=12),
                expansion=hp.Int('expansion_2', min_value=4, max_value=8, step=2),
                use_se=hp.Boolean('use_se_2'),
                dropout_rate=hp.Float('conv6_dropout_rate_2', min_value=0.0, max_value=0.2, step=0.1))
    if hp.Boolean('use_maxpool'):
        x = layers.MaxPool2D()(x)

    x = layers.Conv2D(filters=hp.Int('conv_filter', min_value=32, max_value=96, step=64),
                      kernel_size=hp.Choice('conv_kernel', values=[3,  7]),
                      activation=activation_function, padding='same',
                      kernel_regularizer=keras.regularizers.l2(0.003) if hp.Boolean('l2_reg') else None)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(filters=hp.Int('conv_filter', min_value=32, max_value=96, step=64),
                      kernel_size=hp.Choice('conv_kernel', values=[3,  7]),
                      activation=activation_function, padding='same',
                      kernel_regularizer=keras.regularizers.l2(0.003) if hp.Boolean('l2_reg') else None)(x)
    if hp.Boolean('use_maxpool'):
        x = layers.MaxPool2D()(x)

    x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)
    

    for i in range(hp.Int('num_dense_layers', min_value=4, max_value=6, step=2)):   
        x = layers.Dense(hp.Int(f'dense_units', min_value=512, max_value=1024, step=512),
                         activation=activation_function,
                         kernel_regularizer=keras.regularizers.l2(0.003) if hp.Boolean('l2_reg') else None)(x)
        x = layers.Dropout(hp.Float(f'dropout_rate', min_value=0.0, max_value=0.4, step=0.2))(x)
    
    x = layers.Dense(1, activation='sigmoid')(x)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=0.005, max_value=0.08, sampling='LOG'),
                                         epsilon=0.01)
    model = Model(inputs=input_tensor, outputs=x)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['binary_accuracy'])
    return model


In [ ]:
tuner = Hyperband(
    build_model,
    objective='val_binary_accuracy',
    max_epochs=40,
    factor=2,
    directory='/kaggle/working/projects',
    project_name='cifake_hyperband_1')

In [ ]:
# Start the hyperparameter search with early stopping
tuner.search(ds_train, epochs=40, validation_data=ds_valid, callbacks=[early_stopping, tensorboard, reduce_lr, model_checkpoint])

best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
best_model.summary()

In [ ]:
tuner.results_summary()

In [ ]:
best_model.save(f"/kaggle/working/BEST{NAME}.keras")
